In [1]:
import numpy as np
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, LSTM, Flatten, Bidirectional
from tensorflow.keras.layers import Conv1D, TimeDistributed, MaxPooling1D
from tensorflow.keras.layers import ConvLSTM2D

class UnivariateModels:
    '''
    单变量时间序列LSTM模型
    '''
    def __init__(self, sequence, test_seq, n_seq, n_steps, sw_width, features, epochs_num, verbose_set, flag): 
        self.sequence = sequence
        self.test_seq = test_seq
        self.sw_width = sw_width
        self.features = features
        self.epochs_num = epochs_num
        self.verbose_set = verbose_set
        self.flag = flag
        self.X, self.y = [], []
        
        self.n_seq = n_seq
        self.n_steps = n_steps       
        
    def split_sequence(self):
        for i in range(len(self.sequence)):
            # 找到最后一个元素的索引
            end_index = i + self.sw_width
            # 如果最后一个滑动窗口中的最后一个元素的索引大于序列中最后一个元素的索引则丢弃该样本
            if end_index > len(self.sequence) - 1:
                break
                
            # 实现以滑动步长为1（因为是for循环），窗口宽度为self.sw_width的滑动步长取值
            seq_x, seq_y = self.sequence[i:end_index], self.sequence[end_index]
            self.X.append(seq_x)
            self.y.append(seq_y)
            
        self.X, self.y = np.array(self.X), np.array(self.y)
    
        for i in range(len(self.X)):
            print(self.X[i], self.y[i])
        
        if self.flag == 1:
            self.X = self.X.reshape((self.X.shape[0], self.n_seq, self.n_steps, self.features))
        elif self.flag == 2:
            self.X = self.X.reshape((self.X.shape[0], self.n_seq, 1, self.n_steps, self.features))
        else:
            self.X = self.X.reshape((self.X.shape[0], self.X.shape[1], self.features))
        
        print('X:\n{}\ny:\n{}\n'.format(self.X, self.y))
        print('X.shape:{}, y.shape:{}\n'.format(self.X.shape, self.y.shape))
        return self.X, self.y
    
    def vanilla_lstm(self):
        model = Sequential()
        model.add(LSTM(50, activation='relu', input_shape=(self.sw_width, self.features)))
        model.add(Dense(1))
        model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
        print(model.summary())
        
        history = model.fit(self.X, self.y, epochs=self.epochs_num, verbose=self.verbose_set)
        print('\ntrain_acc:%s'%np.mean(history.history['accuracy']), '\ntrain_loss:%s'%np.mean(history.history['loss']))
        print('yhat:%s'%(model.predict(self.test_seq)),'\n-----------------------------')
        
#         model = Sequential()
#         model.add(LSTM(50, activation='relu', input_shape=(self.sw_width, self.features),
#                        # 其它参数配置
#                        recurrent_activation='sigmoid', use_bias=True, kernel_initializer='glorot_uniform', 
#                        recurrent_initializer='orthogonal', bias_initializer='zeros', unit_forget_bias=True, kernel_regularizer=None, 
#                        recurrent_regularizer=None, bias_regularizer=None, kernel_constraint=None, recurrent_constraint=None, 
#                        bias_constraint=None, dropout=0.0, recurrent_dropout=0.0, implementation=2))
        
#         model.add(Dense(units=1,
#                         # 其它参数配置
#                         activation=None, use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros', 
#                         kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, bias_constraint=None))
        
#         model.compile(optimizer='adam', loss='mse', 
#                       # 其它参数配置
#                       metrics=None, loss_weights=None, sample_weight_mode=None, weighted_metrics=None, target_tensors=None)
#         print(model.summary())
        
#         history = model.fit(self.X, self.y, self.epochs_num, self.verbose_set, 
#                   # 其它参数配置
#                   callbacks=None, validation_split=0.0, validation_data=None, shuffle=True, class_weight=None, sample_weight=None, 
#                   initial_epoch=0, steps_per_epoch=None, validation_steps=None, validation_freq=1, max_queue_size=10, workers=1, use_multiprocessing=False)
                
#         model.predict(self.test_seq, verbose=self.verbose_set,
#                       # 其它参数配置
#                       steps=None, callbacks=None, max_queue_size=10, workers=1, use_multiprocessing=False)

    def stacked_lstm(self):
        model = Sequential()
        model.add(LSTM(50, activation='relu', return_sequences=True, 
                       input_shape=(self.sw_width, self.features)))
        model.add(LSTM(50, activation='relu'))
        model.add(Dense(1))
        model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

        history = model.fit(self.X, self.y, epochs=self.epochs_num, verbose=self.verbose_set)
        print('\ntrain_acc:%s'%np.mean(history.history['accuracy']), '\ntrain_loss:%s'%np.mean(history.history['loss']))
        print('yhat:%s'%(model.predict(self.test_seq)),'\n-----------------------------')
        
    def bidirectional_lstm(self):
        model = Sequential()
        model.add(Bidirectional(LSTM(50, activation='relu'), 
                                input_shape=(self.sw_width, self.features)))
        model.add(Dense(1))
        model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
        
        history = model.fit(self.X, self.y, epochs=self.epochs_num, verbose=self.verbose_set)
        print('\ntrain_acc:%s'%np.mean(history.history['accuracy']), '\ntrain_loss:%s'%np.mean(history.history['loss']))
        print('yhat:%s'%(model.predict(self.test_seq)),'\n-----------------------------')
        
    def cnn_lstm(self):
        model = Sequential()
        model.add(TimeDistributed(Conv1D(filters=64, kernel_size=1, activation='relu'),
                                  input_shape=(None, self.n_steps, self.features)))
        model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
        model.add(TimeDistributed(Flatten()))
        model.add(LSTM(50, activation='relu'))
        model.add(Dense(1))
        model.compile(optimizer='adam', loss='mse',metrics=['accuracy'])
        
        history = model.fit(self.X, self.y, epochs=self.epochs_num, verbose=self.verbose_set)
        print('\ntrain_acc:%s'%np.mean(history.history['accuracy']), '\ntrain_loss:%s'%np.mean(history.history['loss']))
        print('yhat:%s'%(model.predict(self.test_seq)),'\n-----------------------------')
    
    def conv_lstm(self):
        model = Sequential()
        model.add(ConvLSTM2D(filters=64, kernel_size=(1,2), activation='relu',
                            input_shape=(self.n_seq, 1, self.n_steps, self.features)))
        model.add(Flatten())
        model.add(Dense(1))
        model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
        
        history = model.fit(self.X, self.y, epochs=self.epochs_num, verbose=self.verbose_set)
        print('\ntrain_acc:%s'%np.mean(history.history['accuracy']), '\ntrain_loss:%s'%np.mean(history.history['loss']))
        print('yhat:%s'%(model.predict(self.test_seq)),'\n-----------------------------')
        

In [2]:
if __name__ == '__main__':
    single_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]
    sw_width = 3
    features = 1
    
    n_seq = 2
    n_steps = 2
    epochs = 300
    verbose = 0
    
    test_seq = np.array([70, 80, 90])
    test_seq = test_seq.reshape((1, sw_width, features))
    
    UnivariateLSTM = UnivariateModels(single_seq, test_seq, n_seq, n_steps, sw_width, features, epochs, verbose, flag=0)
    UnivariateLSTM.split_sequence()
    UnivariateLSTM.vanilla_lstm()
    UnivariateLSTM.stacked_lstm()
    UnivariateLSTM.bidirectional_lstm()

[10 20 30] 40
[20 30 40] 50
[30 40 50] 60
[40 50 60] 70
[50 60 70] 80
[60 70 80] 90
X:
[[[10]
  [20]
  [30]]

 [[20]
  [30]
  [40]]

 [[30]
  [40]
  [50]]

 [[40]
  [50]
  [60]]

 [[50]
  [60]
  [70]]

 [[60]
  [70]
  [80]]]
y:
[40 50 60 70 80 90]

X.shape:(6, 3, 1), y.shape:(6,)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 50)                10400     
_________________________________________________________________
dense (Dense)                (None, 1)                 51        
Total params: 10,451
Trainable params: 10,451
Non-trainable params: 0
_________________________________________________________________
None

train_acc:0.0 
train_loss:603.0445827874324
yhat:[[100.85666]] 
-----------------------------

train_acc:0.0 
train_loss:337.3305027484894
yhat:[[102.78299]] 
-----------------------------

train_acc:0.0 
train_loss:283.148442

In [3]:
if __name__ == '__main__':
    single_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]
    sw_width = 4
    features = 1
    
    n_seq = 2
    n_steps = 2
    epochs = 500
    verbose = 0
    
    test_seq = np.array([60, 70, 80, 90])
    test_seq = test_seq.reshape((1, n_seq, n_steps, features))
    
    UnivariateLSTM = UnivariateModels(single_seq, test_seq, n_seq, n_steps, sw_width, features, epochs, verbose, flag=1)
    UnivariateLSTM.split_sequence()
    UnivariateLSTM.cnn_lstm()
    
    test_seq = test_seq.reshape((1, n_seq, 1, n_steps, features))
    UnivariateLSTM = UnivariateModels(single_seq, test_seq, n_seq, n_steps, sw_width, features, epochs, verbose, flag=2)
    UnivariateLSTM.split_sequence()
    UnivariateLSTM.conv_lstm()    

[10 20 30 40] 50
[20 30 40 50] 60
[30 40 50 60] 70
[40 50 60 70] 80
[50 60 70 80] 90
X:
[[[[10]
   [20]]

  [[30]
   [40]]]


 [[[20]
   [30]]

  [[40]
   [50]]]


 [[[30]
   [40]]

  [[50]
   [60]]]


 [[[40]
   [50]]

  [[60]
   [70]]]


 [[[50]
   [60]]

  [[70]
   [80]]]]
y:
[50 60 70 80 90]

X.shape:(5, 2, 2, 1), y.shape:(5,)


train_acc:0.0 
train_loss:96.9693284504041
yhat:[[100.737885]] 
-----------------------------
[10 20 30 40] 50
[20 30 40 50] 60
[30 40 50 60] 70
[40 50 60 70] 80
[50 60 70 80] 90
X:
[[[[[10]
    [20]]]


  [[[30]
    [40]]]]



 [[[[20]
    [30]]]


  [[[40]
    [50]]]]



 [[[[30]
    [40]]]


  [[[50]
    [60]]]]



 [[[[40]
    [50]]]


  [[[60]
    [70]]]]



 [[[[50]
    [60]]]


  [[[70]
    [80]]]]]
y:
[50 60 70 80 90]

X.shape:(5, 2, 1, 2, 1), y.shape:(5,)


train_acc:0.0 
train_loss:236.70842473191
yhat:[[103.98932]] 
-----------------------------


In [1]:
import numpy as np

In [4]:
x_input_ = np.array([[80, 85], [90, 95], [100, 105]])
x_input_.shape

(3, 2)

In [1]:
import numpy as np
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, LSTM, Flatten

def mock_seq(seq1, seq2):
    '''
    构造虚拟序列数据
    实现将多个单列序列数据构造成类似于实际数据的列表的格式
    '''
    seq1 = np.array(seq1)
    seq2 = np.array(seq2)
    seq3 = np.array([seq1[i]+seq2[i] for i in range(len(seq1))])
    
    seq1 = seq1.reshape((len(seq1), 1))
    seq2 = seq2.reshape((len(seq2), 1))
    seq3 = seq3.reshape((len(seq3), 1))
    
    # 对于二维数组，沿第二个维度堆叠，相当于列数增加；可以想象成往书架里一本一本的摆书
    dataset = np.hstack((seq1, seq2, seq3))
    
    return dataset
        
    
class MultiInputModels:
    '''
    单变量时间序列LSTM模型
    '''
    def __init__(self, train_seq, test_seq, sw_width, epochs_num, verbose_set): 
        '''
        初始化变量和参数
        '''
        self.train_seq = train_seq
        self.test_seq = test_seq
        self.sw_width = sw_width
        
        self.epochs_num = epochs_num
        self.verbose_set = verbose_set
        
        self.X, self.y = [], []     
        
    def split_sequence_multi_input(self):
        '''
        该函数实现多输入序列数据的样本划分
        '''
        for i in range(len(self.train_seq)):
            # 找到最后一个元素的索引，因为for循环中i从1开始，切片索引从0开始，切片区间前闭后开，所以不用减去1；
            end_index = i + self.sw_width
            # 如果最后一个滑动窗口中的最后一个元素的索引大于序列中最后一个元素的索引则丢弃该样本；
            # 这里len(self.sequence)没有减去1的原因是：保证最后一个元素的索引恰好等于序列数据索引时，能够截取到样本；
            if end_index > len(self.train_seq) :
                break
                
            # 实现以滑动步长为1（因为是for循环），窗口宽度为self.sw_width的滑动步长取值；
            # [i:end_index, :-1] 截取第i行到第end_index-1行、除最后一列之外的列的数据；
            # [end_index-1, -1] 截取第end_index-1行、最后一列的单个数据，其实是在截取期望预测值y；
            seq_x, seq_y = self.train_seq[i:end_index, :-1], self.train_seq[end_index-1, -1]
            self.X.append(seq_x)
            self.y.append(seq_y)
            
        self.X, self.y = np.array(self.X), np.array(self.y)
        self.features = self.X.shape[2]
        self.test_seq = self.test_seq.reshape((1, self.sw_width, self.test_seq.shape[1]))

        for i in range(len(self.X)):
            print(self.X[i], self.y[i])
        
        print('X:\n{}\ny:\n{}\ntest_seq:\n{}\n'.format(self.X, self.y, self.test_seq))
        print('X.shape:{}, y.shape:{}, test_seq.shape:{}\n'.format(self.X.shape, self.y.shape, self.test_seq.shape))
        
        return self.X, self.y, self.features, self.test_seq
    
    def split_sequence_parallel(self):
        '''
        该函数实现多输入序列数据的样本划分
        '''
        for i in range(len(self.train_seq)):
            # 找到最后一个元素的索引，因为for循环中i从1开始，切片索引从0开始，切片区间前闭后开，所以不用减去1；
            end_index = i + self.sw_width
            # 如果最后一个滑动窗口中的最后一个元素的索引大于序列中最后一个元素的索引则丢弃该样本；
            # 这里len(self.sequence)减去1的原因是：保证最后一个元素的索引恰好等于序列数据索引时，能够截取到样本；
            if end_index > len(self.train_seq) - 1:
                break
                
            # 实现以滑动步长为1（因为是for循环），窗口宽度为self.sw_width的滑动步长取值；
            # [i:end_index, :] 截取第i行到第end_index-1行、所有列的数据；
            # [end_index-1, :] 截取第end_index行、所有列的数据；
            seq_x, seq_y = self.train_seq[i:end_index, :], self.train_seq[end_index, :]
            self.X.append(seq_x)
            self.y.append(seq_y)
            
        self.X, self.y = np.array(self.X), np.array(self.y)
        self.features = self.X.shape[2]
        self.test_seq = self.test_seq.reshape((1, self.sw_width, self.test_seq.shape[1]))

        for i in range(len(self.X)):
            print(self.X[i], self.y[i])
        
        print('X:\n{}\ny:\n{}\ntest_seq:\n{}\n'.format(self.X, self.y, self.test_seq))
        print('X.shape:{}, y.shape:{}, test_seq.shape:{}\n'.format(self.X.shape, self.y.shape, self.test_seq.shape))
        
        return self.X, self.y, self.features, self.test_seq    
    
    def vanilla_lstm(self):
        model = Sequential()
        model.add(LSTM(50, activation='relu', 
                       input_shape=(self.sw_width, self.features)))
        model.add(Dense(1))
        model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
        print(model.summary())
        
        history = model.fit(self.X, self.y, epochs=self.epochs_num, verbose=self.verbose_set)
        print('\ntrain_acc:%s'%np.mean(history.history['accuracy']), '\ntrain_loss:%s'%np.mean(history.history['loss']))
        print('yhat:%s'%(model.predict(self.test_seq)),'\n-----------------------------')
        
    def stacked_lstm(self):
        model = Sequential()
        model.add(LSTM(100, activation='relu', return_sequences=True, 
                       input_shape=(self.sw_width, self.features)))
        model.add(LSTM(100, activation='relu'))
        model.add(Dense(self.features))
        model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

        history = model.fit(self.X, self.y, epochs=self.epochs_num, verbose=self.verbose_set)
        print('\ntrain_acc:%s'%np.mean(history.history['accuracy']), '\ntrain_loss:%s'%np.mean(history.history['loss']))
        print('yhat:%s'%(model.predict(self.test_seq)),'\n-----------------------------')
        

In [2]:
if __name__ == '__main__':
    
    orig_seq1 = [10, 20, 30, 40, 50, 60, 70, 80, 90]
    orig_seq2 = [15, 25, 35, 45, 55, 65, 75, 85, 95]
    train_seq = mock_seq(orig_seq1, orig_seq2)
    
    test_seq_multi = np.array([[80, 85], [90, 95], [100, 105]])
    test_seq_paral = np.array([[70,75,145], [80,85,165], [90,95,185]])
    
    sw_width = 3
    epochs_num = 500
    verbose_set = 0
    
    print('-----------以下为 【多输入序列LSTM模型】 相关信息-----------------')
    MultiInputLSTM = MultiInputModels(train_seq, test_seq_multi, sw_width, epochs_num, verbose_set)
    MultiInputLSTM.split_sequence_multi_input()
    MultiInputLSTM.vanilla_lstm()
    print('-----------以下为 【多并行序列LSTM模型】 相关信息-----------------')
    MultiInputLSTM = MultiInputModels(train_seq, test_seq_paral, sw_width, epochs_num, verbose_set)
    MultiInputLSTM.split_sequence_parallel()
    MultiInputLSTM.stacked_lstm()


-----------以下为 【多输入序列LSTM模型】 相关信息-----------------
[[10 15]
 [20 25]
 [30 35]] 65
[[20 25]
 [30 35]
 [40 45]] 85
[[30 35]
 [40 45]
 [50 55]] 105
[[40 45]
 [50 55]
 [60 65]] 125
[[50 55]
 [60 65]
 [70 75]] 145
[[60 65]
 [70 75]
 [80 85]] 165
[[70 75]
 [80 85]
 [90 95]] 185
X:
[[[10 15]
  [20 25]
  [30 35]]

 [[20 25]
  [30 35]
  [40 45]]

 [[30 35]
  [40 45]
  [50 55]]

 [[40 45]
  [50 55]
  [60 65]]

 [[50 55]
  [60 65]
  [70 75]]

 [[60 65]
  [70 75]
  [80 85]]

 [[70 75]
  [80 85]
  [90 95]]]
y:
[ 65  85 105 125 145 165 185]
test_seq:
[[[ 80  85]
  [ 90  95]
  [100 105]]]

X.shape:(7, 3, 2), y.shape:(7,), test_seq.shape:(1, 3, 2)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 50)                10600     
_________________________________________________________________
dense (Dense)                (None, 1)                 51        
Total par

In [3]:
import numpy as np
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, LSTM, Flatten, Bidirectional
from tensorflow.keras.layers import TimeDistributed,RepeatVector
        
    
class MultiStepModels:
    '''
    多时间步预测时间序列LSTM模型
    '''
    def __init__(self, train_seq, test_seq, sw_width, pred_length, features, epochs_num, verbose_set, flag=0): 
        '''
        初始化变量和参数
        '''
        self.train_seq = train_seq
        self.test_seq = test_seq
        self.sw_width = sw_width
        self.pred_length = pred_length
        
        self.features = features
        
        self.epochs_num = epochs_num
        self.verbose_set = verbose_set
        
        self.flag = flag
        
        self.X, self.y = [], []     
        
    def split_sequence(self):
        '''
        该函数实现多输入序列数据的样本划分
        '''
        for i in range(len(self.train_seq)):
            # 找到最后一个元素的索引，因为for循环中i从1开始，切片索引从0开始，切片区间前闭后开，所以不用减去1；
            end_index = i + self.sw_width
            # 找到需要预测指定时间步长的最后一个元素的索引；
            out_end_index = end_index + self.pred_length
            # 如果最后一个期望输出最后一个元素的索引大于序列中最后一个元素的索引则丢弃该样本；
            # 这里len(self.sequence)没有减去1的原因是：保证最后一个元素的索引恰好等于序列数据索引时，能够截取到样本；
            if out_end_index > len(self.train_seq) :
                break
                
            # 实现以滑动步长为1（因为是for循环），窗口宽度为self.sw_width的滑动步长取值；
            seq_x, seq_y = self.train_seq[i:end_index], self.train_seq[end_index:out_end_index]
            self.X.append(seq_x)
            self.y.append(seq_y)
            
        self.X, self.y = np.array(self.X), np.array(self.y)
        self.X = self.X.reshape((self.X.shape[0], self.X.shape[1], self.features))
        self.test_seq = self.test_seq.reshape((1, self.sw_width, self.features))

        if self.flag == 1:
            self.y = self.y.reshape((self.y.shape[0], self.y.shape[1], self.features))
        else:
            pass
        
        for i in range(len(self.X)):
            print(self.X[i], self.y[i])
        
        print('X:\n{}\ny:\n{}\ntest_seq:\n{}\n'.format(self.X, self.y, self.test_seq))
        print('X.shape:{}, y.shape:{}, test_seq.shape:{}\n'.format(self.X.shape, self.y.shape, self.test_seq.shape))
        
        return self.X, self.y, self.test_seq       

    def stacked_lstm(self):
        model = Sequential()
        model.add(LSTM(100, activation='relu', return_sequences=True, 
                       input_shape=(self.sw_width, self.features)))
        model.add(LSTM(100, activation='relu'))
        model.add(Dense(units=self.pred_length))
        model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
        print(model.summary())

        history = model.fit(self.X, self.y, epochs=self.epochs_num, verbose=self.verbose_set)
        print('\ntrain_acc:%s'%np.mean(history.history['accuracy']), '\ntrain_loss:%s'%np.mean(history.history['loss']))
        print('yhat:%s'%(model.predict(self.test_seq)),'\n-----------------------------')

    def encoder_decoder_lstm(self):
        model = Sequential()
        model.add(LSTM(100, activation='relu',
                       input_shape=(self.sw_width, self.features)))
        model.add(RepeatVector(self.pred_length))
        model.add(LSTM(100, activation='relu', return_sequences=True))
        model.add(TimeDistributed(Dense(1)))
        
        model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
        print(model.summary())
        
        history = model.fit(self.X, self.y, epochs=self.epochs_num, verbose=self.verbose_set)
        print('\ntrain_acc:%s'%np.mean(history.history['accuracy']), '\ntrain_loss:%s'%np.mean(history.history['loss']))
        print('yhat:%s'%(model.predict(self.test_seq)),'\n-----------------------------')


In [4]:
if __name__ == '__main__':

    train_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]
    test_seq = np.array([70, 80, 90])
    
    sliding_window_width = 3
    predict_length = 2
    n_features = 1
    
    epochs_num = 100
    verbose_set = 0
    
    print('-------以下为 【向量输出 LSTM 模型】 相关信息------')
    MultiStepLSTM = MultiStepModels(train_seq, test_seq, sliding_window_width, predict_length, n_features, 
                                      epochs_num, verbose_set)
    MultiStepLSTM.split_sequence()
    MultiStepLSTM.stacked_lstm()
    
    print('-------以下为 【编码器-解码器 LSTM 模型】 相关信息------')
    MultiStepLSTM = MultiStepModels(train_seq, test_seq, sliding_window_width, predict_length, n_features, 
                                      epochs_num, verbose_set, flag=1)
    MultiStepLSTM.split_sequence()
    MultiStepLSTM.encoder_decoder_lstm()

-------以下为 【向量输出 LSTM 模型】 相关信息------
[[10]
 [20]
 [30]] [40 50]
[[20]
 [30]
 [40]] [50 60]
[[30]
 [40]
 [50]] [60 70]
[[40]
 [50]
 [60]] [70 80]
[[50]
 [60]
 [70]] [80 90]
X:
[[[10]
  [20]
  [30]]

 [[20]
  [30]
  [40]]

 [[30]
  [40]
  [50]]

 [[40]
  [50]
  [60]]

 [[50]
  [60]
  [70]]]
y:
[[40 50]
 [50 60]
 [60 70]
 [70 80]
 [80 90]]
test_seq:
[[[70]
  [80]
  [90]]]

X.shape:(5, 3, 1), y.shape:(5, 2), test_seq.shape:(1, 3, 1)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 3, 100)            40800     
_________________________________________________________________
lstm_5 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 202       
Total params: 121,402
Trainable params: 121,402
Non-trainable params: 0
__________

In [1]:
import numpy as np
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, LSTM, Flatten
from tensorflow.keras.layers import RepeatVector, TimeDistributed

def mock_seq(seq1, seq2):
    '''
    构造虚拟序列数据
    实现将多个单列序列数据构造成类似于实际数据的列表的格式
    '''
    seq1 = np.array(seq1)
    seq2 = np.array(seq2)
    seq3 = np.array([seq1[i]+seq2[i] for i in range(len(seq1))])
    
    seq1 = seq1.reshape((len(seq1), 1))
    seq2 = seq2.reshape((len(seq2), 1))
    seq3 = seq3.reshape((len(seq3), 1))
    
    # 对于二维数组，沿第二个维度堆叠，相当于列数增加；可以想象成往书架里一本一本的摆书
    dataset = np.hstack((seq1, seq2, seq3))
    
    return dataset
        
    
class MultivariateMultiStepModels:
    '''
    多变量多时间步预测LSTM模型
    '''
    def __init__(self, train_seq, test_seq, sw_width, pred_length, epochs_num, verbose_set): 
        '''
        初始化变量和参数
        '''
        self.train_seq = train_seq
        self.test_seq = test_seq
        self.sw_width = sw_width
        self.pred_length = pred_length
        
        self.epochs_num = epochs_num
        self.verbose_set = verbose_set
        
        self.X, self.y = [], []     
        
    def split_sequence_multi_output(self):
        '''
        该函数实现多输入序列数据的样本划分
        '''
        for i in range(len(self.train_seq)):
            # 找到最后一个元素的索引，因为for循环中i从1开始，切片索引从0开始，切片区间前闭后开，所以不用减去1；
            end_index = i + self.sw_width
            # 找到输出预测序列的最大索引，便于截取数据，因为i从1开始，切片索引从0开始，所以需要减去1；
            out_end_index = end_index + self.pred_length - 1
            # 如果最后一个滑动窗口中的最后一个元素的索引大于序列中最后一个元素的索引则丢弃该样本；
            # 这里len(self.sequence)没有减去1的原因是：保证最后一个元素的索引恰好等于序列数据索引时，能够截取到样本；
            if out_end_index > len(self.train_seq) :
                break
                
            # 实现以滑动步长为1（因为是for循环），窗口宽度为self.sw_width的滑动步长取值；
            # [i:end_index, :-1] 截取第i行到第end_index-1行、除最后一列之外的列的数据；
            # [end_index-1:out_end_index, -1] 截取第end_index-1行到第out_end_index行、最后一列的数据；
            seq_x, seq_y = self.train_seq[i:end_index, :-1], self.train_seq[end_index-1:out_end_index, -1]
            self.X.append(seq_x)
            self.y.append(seq_y)
            
        self.X, self.y = np.array(self.X), np.array(self.y)
        self.features = self.X.shape[2]
        self.test_seq = self.test_seq.reshape((1, self.sw_width, self.test_seq.shape[1]))

        for i in range(len(self.X)):
            print(self.X[i], self.y[i])
        
        print('X:\n{}\ny:\n{}\ntest_seq:\n{}\n'.format(self.X, self.y, self.test_seq))
        print('X.shape:{}, y.shape:{}, test_seq.shape:{}\n'.format(self.X.shape, self.y.shape, self.test_seq.shape))
        
        return self.X, self.y, self.features, self.test_seq
    
    def split_sequence_parallel(self):
        '''
        该函数实现多输入序列数据的样本划分
        注意切片区间的选取！其实记住前闭后开区间就很好理解了。
        '''
        for i in range(len(self.train_seq)):
            end_index = i + self.sw_width
            out_end_index = end_index + self.pred_length
            if out_end_index > len(self.train_seq) :
                break
                
            # [i:end_index, :] 截取第i行到第end_index-1行、所有列的数据；
            # [end_index:out_end_index, :] 截取第end_index行到第out_end_index行、所有列的数据；
            seq_x, seq_y = self.train_seq[i:end_index, :], self.train_seq[end_index:out_end_index, :]
            self.X.append(seq_x)
            self.y.append(seq_y)
            
        self.X, self.y = np.array(self.X), np.array(self.y)
        self.features = self.X.shape[2]
        self.test_seq = self.test_seq.reshape((1, self.sw_width, self.test_seq.shape[1]))

        for i in range(len(self.X)):
            print(self.X[i], self.y[i])
        
        print('X:\n{}\ny:\n{}\ntest_seq:\n{}\n'.format(self.X, self.y, self.test_seq))
        print('X.shape:{}, y.shape:{}, test_seq.shape:{}\n'.format(self.X.shape, self.y.shape, self.test_seq.shape))
        
        return self.X, self.y, self.features, self.test_seq    
    
    def stacked_lstm(self):
        model = Sequential()
        model.add(LSTM(100, activation='relu', return_sequences=True, 
                       input_shape=(self.sw_width, self.features)))
        model.add(LSTM(100, activation='relu'))
        model.add(Dense(units=self.pred_length))
        model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
        print(model.summary())
        
        history = model.fit(self.X, self.y, epochs=self.epochs_num, verbose=self.verbose_set)
        print('\ntrain_acc:%s'%np.mean(history.history['accuracy']), '\ntrain_loss:%s'%np.mean(history.history['loss']))
        print('yhat:%s'%(model.predict(self.test_seq)),'\n-----------------------------')
        
    def encoder_decoder_lstm(self):
        model = Sequential()
        model.add(LSTM(200, activation='relu',
                       input_shape=(self.sw_width, self.features)))
        model.add(RepeatVector(self.pred_length))
        model.add(LSTM(200, activation='relu', return_sequences=True))
        model.add(TimeDistributed(Dense(self.features)))
        model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
        print(model.summary())
        
        history = model.fit(self.X, self.y, epochs=self.epochs_num, verbose=self.verbose_set)
        print('\ntrain_acc:%s'%np.mean(history.history['accuracy']), '\ntrain_loss:%s'%np.mean(history.history['loss']))
        print('yhat:%s'%(model.predict(self.test_seq)),'\n-----------------------------')
        

In [2]:
if __name__ == '__main__':
    
    orig_seq1 = [10, 20, 30, 40, 50, 60, 70, 80, 90]
    orig_seq2 = [15, 25, 35, 45, 55, 65, 75, 85, 95]
    train_seq = mock_seq(orig_seq1, orig_seq2)
    
    test_seq_multi = np.array([[80, 85], [90, 95], [100, 105]])
    test_seq_paral = np.array([[60, 65, 125], [70, 75, 145], [80, 85, 165]])
    
    sliding_window_width = 3
    predict_length = 2
    epochs_num = 200
    verbose_set = 0
    
    print('-------以下为 【多输入多时间步预测输出 LSTM 模型】 相关信息------')
    MultivariateMultiStepLSTM = MultivariateMultiStepModels(train_seq, test_seq_multi, sliding_window_width, predict_length,
                                                            epochs_num, verbose_set)
    MultivariateMultiStepLSTM.split_sequence_multi_output()
    MultivariateMultiStepLSTM.stacked_lstm()
    
    print('-------以下为 【并行输入多时间步预测输出 LSTM 模型】 相关信息------')
    MultivariateMultiStepLSTM = MultivariateMultiStepModels(train_seq, test_seq_paral, sliding_window_width, predict_length,
                                                            epochs_num, verbose_set)
    MultivariateMultiStepLSTM.split_sequence_parallel()
    MultivariateMultiStepLSTM.encoder_decoder_lstm()


-------以下为 【多输入多时间步预测输出 LSTM 模型】 相关信息------
[[10 15]
 [20 25]
 [30 35]] [65 85]
[[20 25]
 [30 35]
 [40 45]] [ 85 105]
[[30 35]
 [40 45]
 [50 55]] [105 125]
[[40 45]
 [50 55]
 [60 65]] [125 145]
[[50 55]
 [60 65]
 [70 75]] [145 165]
[[60 65]
 [70 75]
 [80 85]] [165 185]
X:
[[[10 15]
  [20 25]
  [30 35]]

 [[20 25]
  [30 35]
  [40 45]]

 [[30 35]
  [40 45]
  [50 55]]

 [[40 45]
  [50 55]
  [60 65]]

 [[50 55]
  [60 65]
  [70 75]]

 [[60 65]
  [70 75]
  [80 85]]]
y:
[[ 65  85]
 [ 85 105]
 [105 125]
 [125 145]
 [145 165]
 [165 185]]
test_seq:
[[[ 80  85]
  [ 90  95]
  [100 105]]]

X.shape:(6, 3, 2), y.shape:(6, 2), test_seq.shape:(1, 3, 2)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 3, 100)            41200     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_____